In [1]:
import tensorflow as tf
import numpy as np

## what I need to read from the dataset.
(1950, 512) ->
each 512 line is an input
convert each 512 line into [0,...num_sequences], [i+1, ... num_sequences] => total no. of sequences should be equal to 512. - Task 1

In [2]:
freq = [i for i in range(512)]
freq = np.array(freq)

In [3]:
# freq = np.random.sample((512, 1))
# labels = np.random.randint(0, 4, (512, 1))

In [4]:
freq = freq.reshape(freq.shape[0], 1)

In [5]:
batch_size = 1
num_steps = 16

In [6]:
frequencies = np.zeros((512, 16), dtype=np.float32)
print(frequencies.shape)

(512, 16)


In [7]:
frequencies[1].shape

(16,)

In [8]:
labels = np.random.randint(0, 4, (512, 1))

In [9]:
labels.shape

(512, 1)

In [10]:
# convert into num step sized repeated samples.
def processFrequencies(freq, num_steps):
    for i in range(freq.shape[0]):
        if freq[i:i+num_steps, 0].shape[0] == num_steps:
            frequencies[i, :] = freq[i:i+num_steps, 0]
        else:
#             print(freq[i:i+num_steps, 0].shape[0])
            frequencies[i, :] = np.pad(freq[i:i+num_steps, 0], (0, 16-freq[i:i+num_steps, 0].shape[0]), 'edge')
    print('loading complete')

In [11]:
freq = processFrequencies(freq, 16)

loading complete


In [12]:
print(frequencies.shape)

(512, 16)


In [13]:
frequencies[0]

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15.], dtype=float32)

In [14]:
frequencies[1]

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16.], dtype=float32)

In [15]:
frequencies[-1]

array([511., 511., 511., 511., 511., 511., 511., 511., 511., 511., 511.,
       511., 511., 511., 511., 511.], dtype=float32)

In [16]:
frequencies[0].shape

(16,)

In [17]:
frequencies.shape

(512, 16)

In [18]:
frequencies = np.reshape(frequencies, (frequencies.shape[0], frequencies.shape[1], 1))

## Process labels array from (512, 1) to (512, 4).
Required for utilizing softmax layer.

In [19]:
labels_reshaped = np.zeros((512, 4))

In [20]:
for i in range(labels.shape[0]):
    labels_reshaped[i, labels[i, 0]] = 1

In [21]:
labels_old = labels
labels = labels_reshaped

In [22]:
labels.shape

(512, 4)

In [23]:
labels[0]

array([0., 1., 0., 0.])

In [24]:
labels[5]

array([0., 1., 0., 0.])

## Use the alternate method described in tensorflow - importing data tutorial.

In [25]:
class Input:
    def __init__(self, frequencies, labels):
        self.freq_placeholder = tf.placeholder(frequencies.dtype, frequencies.shape)
        self.labels_placeholder = tf.placeholder(labels.dtype, labels.shape)
        
        self.dataset = tf.data.Dataset.from_tensor_slices((self.freq_placeholder, self.labels_placeholder))
        self.dataset = self.dataset.batch(batch_size)
        self.dataset = self.dataset.prefetch(16)
        
        self.iterator = self.dataset.make_initializable_iterator()
        self.inputs, self.output_labels = self.iterator.get_next()

## model parameters constants.
<font color='red'>Change parameters here for model tuning.</font>

In [26]:
hidden_layer_dimension = 32
number_layers = 2
use_dropout = False
dropout = 0.0
num_layers = 2

## Model

In [27]:
class Model:
    def __init__(self, inputs, output_labels):
        self.init_state = tf.placeholder(tf.float32, [number_layers, 2, batch_size, hidden_layer_dimension])
        state_per_layer_list = tf.unstack(self.init_state, axis=0)
        rnn_state_tuples = tuple([tf.nn.rnn_cell.LSTMStateTuple(state[0], state[1]) for state in state_per_layer_list])

        if use_dropout:
            cell = tf.nn.rnn_cell.DropoutWrapper(cell, output_keep_prob=dropout)
            
        self.state = current_state = np.zeros((num_layers, 2, batch_size, hidden_layer_dimension))

        if num_layers > 1:
            cell_list = [self.createLSTMCells() for _ in range(num_layers)]
            cell = tf.nn.rnn_cell.MultiRNNCell(cell_list, state_is_tuple=True)
        elif num_layers == 1:
            cell = createLSTMCells()

        self.output, self.state = tf.nn.dynamic_rnn(cell, 
                                          inputs,
                                          dtype=tf.float32, 
                                          initial_state=rnn_state_tuples)
        
        # extract the last output for time=num_seq from the output.
        self.output = tf.transpose(self.output, [1, 0, 2])
        self.output = tf.gather(self.output, int(self.output.shape[0]-1))
        
        self.logits = tf.layers.dense(inputs=self.output, units=4)
        self.prediction = tf.nn.softmax(self.logits)
        self.loss = tf.losses.softmax_cross_entropy(self.logits, output_labels)
        
        # calculate accuracy every 10 epochs.
        self.acc, self.accuracy_ops = tf.metrics.accuracy(labels=tf.argmax(output_labels, 1),
                                            predictions=tf.argmax(self.logits, 1)
                                           )
        
    def createLSTMCells(self):
        cell = tf.contrib.rnn.LSTMCell(hidden_layer_dimension, forget_bias=1.0)
        return cell
# # flatten the rnn output to feed into a softmax layer.
# output = tf.reshape(output, [-1, hidden_size])

# # setup the softmax layer.
# softmax_w = tf.Variable(tf.random_uniform([hidden_size, 1], -init_scale, init_scale))
# softmax_b = tf.Variable(tf.random_uniform([1], -init_scale, init_scale))
# logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)

In [28]:
print(batch_size)
current_state = np.zeros((num_layers, 2, batch_size, hidden_layer_dimension))
print(current_state.shape)

1
(2, 2, 1, 32)


In [29]:
with tf.Session() as sess:
    i = Input(frequencies, labels)
    m = Model(i.inputs, i.output_labels)
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    for epoch in range(1):
        sess.run(i.iterator.initializer, feed_dict={i.freq_placeholder: frequencies, i.labels_placeholder: labels})
        accuracy_list = list()
        for i in range(512):
            op1, softmax_op, _, accuracy = sess.run([m.loss, m.prediction, m.state, m.accuracy_ops], 
                                                    feed_dict={m.init_state: current_state})
            accuracy_list.append(accuracy)
        print(sum(accuracy_list) / len(accuracy_list))

0.27211427470319904
